<a href="https://colab.research.google.com/github/chottokun/colaboratory/blob/main/spaCy_learning_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# Spacyのお勉強
# 参考　https://zenn.dev/shikumiya_hata/articles/b18e362e2eae09
!pip install sumy tinysegmenter Janome
!pip install neologdn emoji mojimoji


In [3]:

from janome.analyzer import Analyzer
from janome.charfilter import UnicodeNormalizeCharFilter, RegexReplaceCharFilter
from janome.tokenizer import Tokenizer as JanomeTokenizer  # sumyのTokenizerと名前が被るため
from janome.tokenfilter import POSKeepFilter, ExtractAttributeFilter
import re

from sumy.parsers.plaintext import PlaintextParser
from sumy.nlp.tokenizers import Tokenizer
from sumy.summarizers.lex_rank import LexRankSummarizer

import emoji
import mojimoji
import neologdn

def preprocessing(text):
    text = re.sub(r'\n', '', text)
    text = re.sub(r'\r', '', text)
    text = re.sub(r'\s', '', text)
    text = text.lower()
    text = mojimoji.zen_to_han(text, kana=True)
    text = mojimoji.han_to_zen(text, digit=False, ascii=False)
    text = ''.join(c for c in text if c not in emoji.UNICODE_EMOJI)
    text = neologdn.normalize(text)

    return text


In [4]:
contents= '''
今日のわれわれの勝利が、政党の勝利ではなく自由の勝利だということを祝おう。始まりとともに終わりを象徴し、変化とともに再建を示している。というのも、二百年近く前にわれわれの祖先が定めたものと全く同じ厳粛な誓いを、みなさんと全能なる神の前で、私は誓ったばかりであるからだ。
世界は、大きく変貌している。人類は、あらゆる形の貧困とあらゆる形の人間の生命を根絶させる力を手にした。だがしかし、われわれの祖先がそのために血をながしてきた独立戦争の信念は、いまだに世界中で論争の的になっている。人間の権利は国家が気前よくくれたものなどではなく、神の手によって授けられているという信念のことだ。われわれは今日、その最初の独立戦争を受け継ぐものであることを忘れてはならない。
今、この時、この場所からこの言葉を伝えようではないか、敵味方を問わずに。たいまつはアメリカの新しい世代にひきつがれたと。その世代は、今世紀に生まれ、戦争にさらされ、厳しく苦い平和にきたえられ、われわれの祖先の遺産を誇りに思い、母国でも世界中でもこの国がいつも守ってきて、今日われわれが守っているこれらの人権が、昔のように取り消されるようなことを目にしたり、認めたりしない世代である。
敵であろうが味方であろうが、全ての国に知らせよう。われわれは自由が生きのび、勝利をおさめるためなら、どんな対価も支払い、どんな重圧にも耐え、どんな困難にも立ち向かい、味方を支持し、敵には対抗するということを。われわれはこのことを固く誓うし、それ以上の用意さえある。
文化と精神的な起源を一にする古くからの同盟国に対して、われわれは信頼にたる友としての忠誠を誓う。われわれが一致団結すれば、なしえないことはほとんどないし、たもとを分かつとすれば、何事もなしとげられない。互いに争いばらばらになって、どうやって強力な挑戦に立ち向かえばいいというのだろう。自由主義世界への仲間入りをよろこぶ新しい国々には、このことを誓おう。植民地支配という一つの形が終わって、単により厳しい鉄の専制が置きかわるわけではないことを。これらの国々が常にわれわれの見解に賛同してくれるとは、期待していない。ただ、これらの国々が自国なりの自由を見出してくれることをいつも強く望みたい。過去には、こういうことがあったのも覚えておいてほしい、つまり愚かにも虎の威をかり、権力を志向するものは、けっきょく自滅するということを。地球の大部分を占めるほったて小屋や村落に住む人々で、みんなが悲惨な足かせをはずせるように苦闘する人々には、その自助努力に対して、どれほどの時間がかかろうとも、最大限の力を貸すことを誓おう。共産主義者がそうするかもしれないからでも、かれらの票が目当てなわけでもない。正しいことだからだ。もし自由な社会が貧しい大多数のものを救えないようであれば、少数の金持ちも救うことはできない。
われわれと密接な関係にある国境の南の国々に対しては、特別な約束をしよう。言葉を実行に移し、進歩のために新しい同盟をむすび、自由な人々や国々が貧困のくさりから解き放たれる手助けをすると。しかしこの平和的な革命の希望が、敵対する勢力のえじきになるようなことがあってはならない。全ての近隣諸国は、われわれがアメリカ大陸のどこにおいても、他国への侵略や政府の転覆といったことに協力して対抗するということを知っておいてほしい。そしてその他の国々も、この地域においてはわれわれは、自分のことは自分でするということを知っておいてもらいたい。
世界中の独立国の集まり、つまり国際連合に対しては、支持の誓いを新たにする。国際連合は、戦争という手段が平和という手段よりはるかに利用された時代においては、われわれの最後の、そして最大の希望である。そしてわれわれは国際連合を単にののしりあいの場にせず、新しく弱小な国への保護を強め、国際憲章のおよぶ地域を拡大するのを支持する誓いを新たにする。
最後に、われわれに敵対しようとする国々に対しては、誓いではなく要求をすることとしよう。両陣営で新たに平和の探求に着手しようという要求を。科学によって束縛をとかれた暗黒たる破壊力が、計画的にあるいは偶然に、全人類を自己破壊の渦にのみこむ前に。われわれは、敵対する国々に弱みをみせて挑発してはならない。われわれの戦力が疑いなく十分であるときのみ、われわれはその戦力を利用することが決してないことを疑うことなく確信できる。大きくそして力をもつ２つの陣営は、この現在の路線を歩むかぎりどちらも安心できない。両陣営は現代兵器の過大なコストに苦しみ、死をもたらす原子力が着実に拡散していくのにまさに恐怖をおぼえながらも、人類の最終戦争を押しとどめている不確かな恐怖のバランスをくずそうと互いに競争しているのである。
われわれは、着手しなければならない、両陣営とも礼儀正しいふるまいは弱さのしるしではなく、誠実さは常に証明しなければならないということを念頭におこう。恐怖ゆえに交渉してはならない、ただ、交渉することを恐れてもいけない。両陣営で、われわれを分裂させている諸問題を言い争う代わりに、何がわれわれを団結させるのか模索しようではないか。まず第一に、両陣営で武器の査察と管理についての、真摯で厳密な提案を練り上げようではないか。そして、お互いの国々を崩壊させるような絶対的な力を、全ての国の完全な管理下におこうではないか。科学の恐怖ではなく、科学の驚異に訴えかけよう。われわれは共に宇宙を探索し、砂漠を征服し、疫病を根絶し、深海を開発し、芸術や商業を振興しよう。両陣営とも地球上の全ての人の心に、イザヤの言葉を宿らせよう。「重荷を下ろし．．．虐げられたものを自由にしよう」
そしてもし協力の足がかりが、疑惑のうずまくジャングルを押し戻すことができたなら、両陣営は新しい力のバランスではなく、新しい法に基づいた世界を協力して作りあげることができるだろう。新しい法に基づいた世界では、強者も公正であり、弱者が保護され、平和が保たれる。
これは、これからの100日でなしとげられることではないだろう。いやこれからの1000日でも、なしとげることもできないかもしれない。この政権の間に、もしくはこの地球上でのわれわれの生きている間でさえなしとげられないかもしれない。しかし着手しようではないか。
われわれのとる道が最終的に成功するか失敗するかは、わたし以上に、あなたがた市民の手にかかっている。この国の建国以来、アメリカ人の各世代は国家に対する忠誠を証明するために召集されてきた。その召集に応えた若いアメリカ人の墓は世界中にある。今トランペットの音がわれわれを再び召集している。武器は必要だが、武器をとれという召集ではない、戦ってはいるが、戦うための召集ではない、長い夜明け前の闘争の重荷を肩に背負えという召集なのである。いつも希望をもって喜びを抱き、苦難に耐えながら、人類の共通の敵、専制、貧困、疫病、そして戦争そのものに対して闘うという重荷を。これらを敵にして、北も南も、東も西も、壮大な世界的な同盟をわれわれは作れないものだろうか？　その同盟は、全人類により実りある生活を保証してくれるだろう。あなたがたもこの歴史的な苦闘に身を投じてみようではないか？
世界の長い歴史の中で、自由が最大の危機にさらされている時に、自由を守る役割を与えられてきた世代はごく少ない。私は、この責任からしりごみするものではないし、むしろ歓迎する。われわれの誰かが自分の立場を、他の人もしくは他の世代と交換するだろうなどということを私は信じない。こうした努力にわれわれが捧げるエネルギー、信念、献身こそがわれわれの国家を、そして国家につかえるわれわれを照らしだすのである。そしてその明かりから発せられる輝きこそが、本当に世界を照らしだすのである。
そして、わが同胞のアメリカ人よ、あなたの国家があなたのために何をしてくれるかではなく、あなたがあなたの国家のために何ができるかを問おうではないか。わが同胞の世界の市民よ、アメリカがあなたのために何をしてくれるかではなく、われわれと共に人類の自由のために何ができるかを問おうではないか。
最後に、あなたがアメリカ市民であろうが、世界の市民であろうが、われわれがあなたに求めるのと同じ高い水準の力と犠牲をここのわれわれに求めて欲しい。良心を唯一のたしかな報酬とみなし、歴史がわれわれの行動に最終的な判断を下してくれることを信じて、神の祝福と助けをもとめながらも、この地球上では神の仕事はわれわれ自身でなしとげなければならないということを肝に銘じて、われわれの愛すべき国を導くために前進しよう。

'''
# J．F ケネディ 大統領就任
# 青空文庫より

text = re.findall("[^。]+。?", preprocessing(contents))

# 形態素解析(単語単位に分割する)
analyzer = Analyzer(
    char_filters=[UnicodeNormalizeCharFilter(),
                  RegexReplaceCharFilter(r'[(\)「」、。]', ' ')],
    tokenizer=JanomeTokenizer(),
    token_filters=[POSKeepFilter(['名詞', '形容詞', '副詞', '動詞']),
                  ExtractAttributeFilter('base_form')])

corpus = [' '.join(analyzer.analyze(sentence)) + u'。' for sentence in text]
# print(corpus)
print(len(corpus))

# 文書要約処理実行
parser = PlaintextParser.from_string(''.join(corpus), Tokenizer('japanese'))

# LexRankで要約を原文書の3割程度抽出
summarizer = LexRankSummarizer()
summarizer.stop_words = [' ']

# 文書の重要なポイントは2割から3割といわれている?ので、それを参考にsentences_countを設定する。
# summary = summarizer(document=parser.document, sentences_count=int(len(corpus)/10*3))
summary = summarizer(document=parser.document, sentences_count=5)
print(u'文書要約結果')
print(len(summary))
for sentence in summary:
  print(text[corpus.index(sentence.__str__())])

71
文書要約結果
5
しかしこの平和的な革命の希望が、敵対する勢力のえじきになるようなことがあってはならない。
最後に、われわれに敵対しようとする国々に対しては、誓いではなく要求をすることとしよう。
われわれは、着手しなければならない、両陣営とも礼儀正しいふるまいは弱さのしるしではなく、誠実さは常に証明しなければならないということを念頭におこう。
「重荷を下ろし...虐げられたものを自由にしよう」そしてもし協力の足がかりが、疑惑のうずまくジャングルを押し戻すことができたなら、両陣営は新しい力のバランスではなく、新しい法に基づいた世界を協力して作りあげることができるだろう。
この国の建国以来、アメリカ人の各世代は国家に対する忠誠を証明するために召集されてきた。
